In [1]:
import numpy as np
import pandas as pd
import random

from sklearn.cross_validation import train_test_split
from sklearn import preprocessing, linear_model, metrics, grid_search

import matplotlib.pylab as plt
%matplotlib inline

np.random.seed(32)
random.seed(32)

/usr/lib64/python2.7/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
data_path = '/nfs/science-tuk-ins/tesco_insurance/adamho/AppliedMLTraining/data/'

## Exercise 5: Logistic Regression (and some data cleaning)

In the last few excercises, we've learnt how to build regression models so that they generalise well to the future.

But what happens when we just to model/predict an event? 

_The Brief_:

It's December 2015. Your client (Ocset) is planning their marketing strategy for Pancake Day in the UK (Feb 28th). They want to target their next round of pancake mailers to customers that are likely to purchase pancakes (they're tired of sending mail to customers that aren't interested). You've had the very smart idea to build a propensity model for your client...

To build the propensity model, you've decided that you need to gather transactional data on customers that did and didn't buy pancakes at the same time *this* year (around Feb 28th 2015). Knowing that history has a tendency of repeating itself, you assume that - if you can build a decent model to predict people's purchases this year - it should also work when making predictions for next year.

Given that purchasing pancake mixes is a binary event (i.e. you either buy it or you don't), you will build a Logistic Regression model.

Your colleague - who has built a similar model in the past - suggested that you use the following features:

1. Spend/Units/Coupons Redeemed
1. Across the whole retailer
1. Across the 'pancake mixes' merchandising group
1. Across the 'pasta' merchandising group
1. Across the 'syrups' merchandising group
1. Across the 'pasta sauces' merhcandising group

Your training set contains these features for customers between January 2014 and January 2015. Your target variable is 'bought_pancakes'; a binary flag that states whether customers bought pancakes in February 2015. 

You have generated those features in your favourite database and are now ready to build your model. 

First, let's read in our new classification dataset. This contains all of the January 2014-2015 data and the target generated from February 2015:

In [3]:
train = pd.read_csv(data_path + 'classification.csv') 

In [4]:
train.head()

household  dollar_sales  units  coupon  unique_upc  unique_store  \
0          1         10.48    7.0     0.0         7.0           2.0   
1          2        130.44   57.0     0.0        28.0           2.0   
2          3          5.28    2.0     0.0         2.0           1.0   
3          4           NaN    NaN     NaN         NaN           NaN   
4          5          9.83    6.0     0.0         4.0           1.0   

   unique_basket  unique_brand  dollar_sales_pancake mixes  \
0            5.0           4.0                        3.78   
1           37.0          15.0                        6.87   
2            1.0           2.0                        2.29   
3            NaN           NaN                         NaN   
4            3.0           4.0                        2.25   

   dollar_sales_pasta       ...         units_pasta  units_pasta sauce  \
0                0.79       ...                 1.0                2.0   
1               42.88       ...                28.0               18.0   
2                 NaN       ...                 NaN                NaN   
3                 NaN       ...                 NaN                NaN   
4                6.08       ...                 4.0                NaN   

   units_syrups  coupon_pancake mixes  coupon_pasta  coupon_pasta sauce  \
0           2.0                   0.0           0.0                 0.0   
1           8.0                   0.0           0.0                 0.0   
2           1.0                   0.0           NaN                 NaN   
3           NaN                   NaN           NaN                 NaN   
4           1.0                   0.0           0.0                 NaN   

   coupon_syrups  lifestyle_high  segmentation  bought_pancakes  
0            0.0              LA           2.0                1  
1            0.0              MM           3.0                1  
2            0.0              MM           0.0                0  
3            NaN              UM           0.0                0  
4            0.0              UM           0.0                0  

[5 rows x 23 columns]

#### Data Preparation
Looking at our data above, I can see that there are some missing data (i.e. NaN). Let's fill that in with 0s.

In [5]:
train = train.fillna(0)

I can also see that there's a categorical (i.e contains string) variable to the right of the dataset called 'lifestyle_high'
Let's create dummies for all variables that contain string using the pd.get_dummies() function

In [6]:
train = pd.get_dummies(train)

Let's separate our X from our y in both datasets and create some random test sets

In [7]:
# Separate out targets, features and ids
train_y = train['bought_pancakes'] # y variables
train_X = train.drop(['household', 'segmentation', 'bought_pancakes'], axis=1) # X features

# Create a training and two test sets
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.7)
test_X, eval_X, test_y, eval_y = train_test_split(test_X, test_y, test_size=0.5)

Looking at the shape of the training set,  I'd also probably take a sample for model development

In [8]:
_, train_X_smp, _, train_y_smp = train_test_split(train_X, train_y, test_size=0.5)
print ('The shape of our training set is now {0:s} ') .format(train_X_smp.shape)

The shape of our training set is now (76504, 22) 


In [9]:
print ('We have {0:d} features in our dataset') .format(train_X.shape[1])

We have 22 features in our dataset


In [10]:
# Let's now scale our data using the MinMaxScaler
scl = preprocessing.MinMaxScaler()
train_X_smp_scl = scl.fit_transform(train_X_smp)
test_X_scl = scl.transform(test_X)
eval_X_scl = scl.transform(eval_X)

#### Modelling

Now that we've preprocessed our data, we need to fit our Logistic Regression. 

Using an 'l1' or 'l2' penalty term is simply as easy as selecting 'l1' or 'l2' as the penalty argument in the LogisticRegression() class.

The $\lambda$ (lambda) parameter can be tuned via the 'C' argument within the LogisticRegression class. C is equivalent to 1-$\lambda$.

Recall that we need to perform a grid-search with cross-validation to find the best performing model. Because the dataset is quite large, we'll perform 3-fold CV for each value of $\lambda$

We'll evaluate our model using Log Loss and AUC. If you want to know more about how AUC is calculated, you can go to this link:
https://www.quora.com/Machine-Learning-What-is-an-intuitive-explanation-of-AUC
An intuitive explanation is: 1.0 = perfect model, 0.5 = random guessing


In [36]:
# Leave this cell empty and return back to it for Question 1

In [12]:
parameters = {'C':[0.1, 0.3, 0.6, 0.9, 1.0], 'penalty' : ['l1']} # The 'C' parameter is 1 - regularisation lambda
mod = linear_model.LogisticRegression()
clf = grid_search.GridSearchCV(mod, parameters, cv=3, n_jobs=1, verbose=3, scoring ='roc_auc')
clf.fit(train_X_smp_scl, train_y_smp) # This performs a grid search across all listed values for C
# Now we can make predictions with the best model from the grid search
preds = clf.predict_proba(test_X_scl)[:, 1]

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] penalty=l1, C=0.1 ...............................................
[CV] ...................... penalty=l1, C=0.1, score=0.641088 -   0.7s
[CV] penalty=l1, C=0.1 ...............................................
[CV] ...................... penalty=l1, C=0.1, score=0.635270 -   1.1s
[CV] penalty=l1, C=0.1 ...............................................
[CV] ...................... penalty=l1, C=0.1, score=0.637714 -   0.4s
[CV] penalty=l1, C=0.3 ...............................................
[CV] ...................... penalty=l1, C=0.3, score=0.654536 -   3.2s
[CV] penalty=l1, C=0.3 ...............................................
[CV] ...................... penalty=l1, C=0.3, score=0.648593 -   2.7s
[CV] penalty=l1, C=0.3 ...............................................
[CV] ...................... penalty=l1, C=0.3, score=0.649627 -   1.5s
[CV] penalty=l1, C=0.6 ...............................................
[CV] ............

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.2min finished


In [13]:
# Now let's evaluate our model
auc = metrics.roc_auc_score(test_y, preds)
logloss = metrics.log_loss(test_y, preds)


print('The Log Loss for this model on the test set is {0:.2f} ') .format(logloss)
print('The AUC for this model on the test set is {0:.2f} ') .format(auc)
print('The best performing C value was {0:s}') .format(clf.best_params_ )

The Log Loss for this model on the test set is 0.41 
The AUC for this model on the test set is 0.66 
The best performing C value was {'penalty': 'l1', 'C': 1.0}


##### Q1: At the moment, we're using a MinMaxScaler to fix all of our features to have a min of 0 and a max of 1. Go back to the cell where we pre-process our data with MinMaxScaler and replace the function with StandardScaler. Then re-run the LogisticRegression() grid-search. Does the test set performance improve or not?

##### Q2: Try fitting your model without any scaling. What happens to the quality of the model? What happens to the speed of the fit when it's performed on unscaled data?

## Multi-class Logistic Regression

So far, we've only considered modelling events with binary outcomes (i.e. purchase/no purchase, cancer/no cancer etc.)

You may also wish to build a model that can predict one of $k$ mutually exclusive events. For this, we need a multi-class classifier.

Your client recently conducted an attitudinal survey on their customers. Following some analysis by their researchers, they created an attitudinal segmentation. After reviewing the attitudinal segmentation, they decided that their customers fall in to one of 5 clusters; Hungry Family (0), Health Conscious (1), All Day Breakfast (2), Thirsty Students (3) or Consciously Clean (4).

They have now asked you - using behavioural data only - to see if you can identify these customers. IF the model works, they would like you to implement the algorithm in production, such that it scores all 18 million of their customers every week.

You have generated some behavioural data using your favourite data warehouse. It describes some shopping behaviour features for 52 weeks up to 4 weeks ago. The segmentation was created within the last month; you've agreed with your client that you don't want your behavioural data overlapping with the time that the segmentation was conducted.

You have decided to build a Multi-class Logistic Regression. You have told your client that it will be quick to run, easy to implement and will provide an explicit equation at the end.

I now need to perform a very similar set of preprocessing steps to the previous project. This should ring alarm bells to any programmer. You should never have to repeat code! Instead, the best way to program is to define functions that perform repetitive tasks for us. That way, we will save time in the long run.

Let's define some functions that will pre-process any training and test set for us...

In [14]:
def fill_missing(dataframe, value):
    """Fill missing values of a dataframe with value"""
    return dataframe.fillna(value)

def _determine_categoricals(dataset):
    """Return list of names for categorical columns in dataframe"""
    num_cols = train._get_numeric_data().columns
    return list(set(train.columns) - set(num_cols))
    
def create_dummy_variables(train, test):
    """Create dummmy variables for categorical columns in train and test (both must be dataframes)"""
    
    # Ensure that the columns are the same on both datasets
    assert(sum(train.columns == test.columns) == len(train.columns))
    
    # Temporarily combine the training and test sets
    full = pd.concat([train.reset_index(drop=True), test.reset_index(drop=True)], axis=0).reset_index(drop=True)
    
    # Determine the categorical columns
    cat_cols = _determine_categoricals(full)
    print('Dummies will be created for the following columns: {0:s}' .format(cat_cols))
    
    # Create dummy variables for those columns
    dummies_full = pd.get_dummies(full)

    # Return the full train and test sets with dummies inside
    train_wdummies = dummies_full.ix[0:train.shape[0]-1,:]
    test_wdummies = dummies_full.ix[train.shape[0]:dummies_full.shape[0],:]
    
    return train_wdummies, test_wdummies

def preprocess_data(train, test):
    """Preprocess a training and test set"""
    
    train = fill_missing(train, 0)
    test = fill_missing(test, 0)
    
    train, test = create_dummy_variables(train, test)
    
    return train, test

In [25]:
# We will just use data from this year (we're not trying to predict next year)
train = pd.read_csv(data_path + 'segmentation.csv') # This is the data from last month

# Separate out targets, features and ids
train_y = train['segmentation'] # y variables
train_X = train.drop(['household', 'segmentation'], axis=1) # X features

# Create a training and two test sets
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.7)
test_X, eval_X, test_y, eval_y = train_test_split(test_X, test_y, test_size=0.5)

# Preprocess the data
_, test_X = preprocess_data(train_X, test_X)
train_X, eval_X = preprocess_data(train_X, eval_X)

Dummies will be created for the following columns: ['lifestyle_high']
Dummies will be created for the following columns: ['lifestyle_high']


Let's reduce the dataset size again and scale the data...

In [26]:
_, train_X_smp, _, train_y_smp = train_test_split(train_X, train_y, test_size=0.5)
print ('The shape of our training set is now {0:s} ') .format(train_X_smp.shape)

# Let's now scale our data using the MinMaxScaler
scl = preprocessing.MinMaxScaler()
train_X_smp_scl = scl.fit_transform(train_X_smp)
test_X_scl = scl.transform(test_X)
eval_X_scl = scl.transform(eval_X)

The shape of our training set is now (76504, 19) 


#### Modelling

Now that we've processed our data, it's time to build a multi-class Logistic Regression. Recall that we are trying to predict segments that were built using an attitudinal survey from behavioural data. We're using the same behavioural data as before. However - rather than trying to predict sales from the following year - we can build our model on a single year's worth of data (i.e. a snapshot of data).

Multi-class logistic regression is very easy to implement using sklearn. In fact, you can use the exact same class as before (i.e. LogisticRegression). Thus, the code is *exactly* the same as before

Again, we'll jump straight in to using L1 regression (because it's amazing) and performing grid search over different levels of $\lambda$.

In [30]:
parameters = {'C':[0.5, 1.0], 'penalty' : ['l1']} # The 'C' parameter is 1 - regularisation lambda
mod = linear_model.LogisticRegression() # Return to this line for Question 5
clf = grid_search.GridSearchCV(mod, parameters, cv=2, n_jobs=2, verbose=3)
clf.fit(train_X_smp_scl, train_y_smp) # This performs a grid search across all listed values for C
# Now we can make predictions with the best model from the grid search
preds = clf.predict_proba(test_X_scl)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] penalty=l1, C=0.5 ...............................................
[CV] ...................... penalty=l1, C=0.5, score=0.910253 -  52.1s
[CV] penalty=l1, C=0.5 ...............................................
[CV] ...................... penalty=l1, C=0.5, score=0.907325 -  51.5s
[CV] penalty=l1, C=1.0 ...............................................
[CV] ...................... penalty=l1, C=1.0, score=0.940212 - 1.2min
[CV] penalty=l1, C=1.0 ...............................................
[CV] ...................... penalty=l1, C=1.0, score=0.937938 - 1.0min


[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:  3.9min finished


In [31]:
logloss = metrics.log_loss(test_y, preds)
acc = metrics.accuracy_score(test_y, clf.predict(test_X_scl))
print('The Log Loss for your model is {0:.2f} ') .format(logloss)
print('The average accuracy of this model across all of the classes was {0:.2f} ') .format(acc)
print('The best performing C value was {0:s}') .format(clf.best_params_ )

The Log Loss for your model is 0.29 
The average accuracy of this model across all of the classes was 0.95 
The best performing C value was {'penalty': 'l1', 'C': 1.0}


Recall from the slides that a multi-class logistic regression can output a matrix of probabilities; one column pertaining to each class in your target. 

When you call the 'predict_proba()' method on your classifier, a matrix of probabilities will be returned.

Alternatively, if you would like to get the predicted class for each row, you can call the 'predict()' method.

##### Q3: Inspect the 'preds' object. In what scenarios can you imagine it being useful to output probabilities for each class? Do you understand what each of these probabilities represent? When would it be better to output the predicted class for each row?

In addition to evaluation metrics, it can be useful to look at a confusion matrix to evaluate the performance of our model.

In a confusion matrix, we plot the predicted values on the horizontal axis and the actual values on the vertical axis. A perfect model will predict each class with 100% accuracy. Thus - in a confusion matrix - you should see more predictions across the diagonal of your matrix.

In the plot below, we calculate - for every level of the segmentation - the proportion that we predicted would be 1,2,3,4 or 5.

In [32]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
act_preds = clf.predict(test_X_scl)
cm = metrics.confusion_matrix(test_y, act_preds)
cm = [cm[:, x] / float(sum(cm[:, x])) for x in range(cm.shape[1])]
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cm)

Looking at the confusion matrix, I would be quite happy with this. Most of the blue is going down the diagonal, which is where we want it to be. It's not perfect, but it's OK.

##### Q4: Go back to the cell where we perform a grid-search with LogisticRegression(). Add the argument class_weight='balanced' inside of the LogisticRegression class and re-run the model. Create the confusion matrix again.

The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as `n_samples / (n_classes * np.bincount(y))`

It can be very useful to use this functionality when you have unbalanced classes. In this modelling problem, we have very unbalanced classes; you can see this by running ```plt.hist(test_y, bins=100);```

What should be done for this modelling problem? In my opinion, I would choose to use the 'balanced' sample_weights option. This is because the confusion matrix and accuracy seems to be much better with balanced sample weights.